#### Step 1: Import Required Libraries

In [1]:
import requests
import pandas as pd
from datetime import datetime
import openai


#### Step 2: Fetch News

In [23]:
# Function to fetch news data from the API
def fetch_news(api_url, query_params):
    try:
        response = requests.get(api_url, params=query_params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching news: {e}")
        return None

# Fetching news from the last 7 days with a limit of 10 results
news_api_url = "https://newsapi.org/v2/everything"
query_params = {
    "q": " supply chain problems",  # Query term
    "from": (datetime.now() - pd.Timedelta(days=7)).strftime('%Y-%m-%d'),  
    "sortBy": "publishedAt",  # Sort by publication date
    #"pageSize": 10,  # Limit results to 10
    "apiKey": "784579032d984b74a00fe6a960512b2c"  
}

news_data = fetch_news(news_api_url, query_params)

if news_data and news_data['totalResults'] > 0:
    print(f"News data fetched successfully!")
else:
    print("No articles found. Try adjusting the query.")


News data fetched successfully!


In [24]:
news_data


{'status': 'ok',
 'totalResults': 117,
 'articles': [{'source': {'id': None, 'name': 'Technewsworld.com'},
   'author': 'TechNewsWorld',
   'title': 'AI Dominates 2025 Cybersecurity Predictions',
   'description': 'Experts predict new threats, expanded attack surfaces, and the critical need for secure and responsible AI adoption as it reshapes cybersecurity strategies in 2025. The post AI Dominates 2025 Cybersecurity Predictions appeared first on TechNewsWorld.',
   'url': 'https://www.technewsworld.com/story/ai-dominates-2025-cybersecurity-predictions-179517.html',
   'urlToImage': 'https://www.technewsworld.com/wp-content/uploads/sites/3/2025/01/AI-thinking.jpg',
   'publishedAt': '2025-01-07T13:00:22Z',
   'content': 'When it comes to cybersecurity in 2025, artificial intelligence is top of mind for many analysts and professionals.\r\nArtificial intelligence will be deployed by both adversaries and defenders, but at… [+9021 chars]'},
  {'source': {'id': None, 'name': 'Forbes'},
   '

####  Aggregate the News Data

In [25]:
# Aggregating the news data into a structured format (DataFrame)
def aggregate_data(news_data):
    try:
        structured_data = []
        for article in news_data['articles']:
            structured_data.append({
                "source": article['source']['name'],
                "title": article['title'],
                "description": article['description'],
                "content": article['content'],
                "published_at": article['publishedAt']
            })
        return pd.DataFrame(structured_data)
    except Exception as e:
        print(f"Error structuring data: {e}")
        return None

# Aggregate data into structured format
structured_data = aggregate_data(news_data)

# Display the first few rows of the structured data
if structured_data is not None:
    print(structured_data.head())
else:
    print("Failed to structure data.")


                source                                              title  \
0    Technewsworld.com        AI Dominates 2025 Cybersecurity Predictions   
1               Forbes  Instant Insights: How To Harness The Power Of ...   
2               Forbes  How The Tea Industry Can Create More Sustainab...   
3  Securityaffairs.com  Moxa router flaws pose serious risks to indust...   
4  Nakedcapitalism.com  NYT Readies Terrain for Incoming Trump Adminis...   

                                         description  \
0  Experts predict new threats, expanded attack s...   
1  Making real-time data an effective part of you...   
2  Whether you're running a tea business or aimin...   
3  Moxa warns of two flaws in its routers and sec...   
4  The media's role in selling the next military ...   

                                             content          published_at  
0  When it comes to cybersecurity in 2025, artifi...  2025-01-07T13:00:22Z  
1  Don Murray is cofounder of Safe Software an

In [26]:
structured_data

,source,title,description,content,published_at
0,Technewsworld.com,AI Dominates 2025 Cybersecurity Predictions,"Experts predict new threats, expanded attack s...","When it comes to cybersecurity in 2025, artifi...",2025-01-07T13:00:22Z
1,Forbes,Instant Insights: How To Harness The Power Of ...,Making real-time data an effective part of you...,Don Murray is cofounder of Safe Software and h...,2025-01-07T13:00:00Z
2,Forbes,How The Tea Industry Can Create More Sustainab...,Whether you're running a tea business or aimin...,Ivan Terzi is the Co-Founder &amp; CEO of TeaD...,2025-01-07T12:45:00Z
3,Securityaffairs.com,Moxa router flaws pose serious risks to indust...,Moxa warns of two flaws in its routers and sec...,US adds Tencent to the list of companies suppo...,2025-01-07T12:27:07Z
4,Nakedcapitalism.com,NYT Readies Terrain for Incoming Trump Adminis...,The media's role in selling the next military ...,The media’s role in selling the next military ...,2025-01-07T11:45:20Z
...,...,...,...,...,...
95,Lesswrong.com,6. Uphold Voluntarism: Digital Defense,"Published on January 2, 2025 7:05 PM GMTSecuri...",Security in the physical realm is an extraordi...,2025-01-02T19:05:33Z
96,Securityaffairs.com,DoubleClickjacking allows clickjacking on majo...,The “DoubleClickjacking” exploit bypasses prot...,DoubleClickjacking allows clickjacking on majo...,2025-01-02T17:34:26Z
97,road.cc,“The bike industry will not get better this ye...,The folding bike brand’s profits fell from £10...,The challenges faced by the bike industry over...,2025-01-02T16:32:10Z
98,Plos.org,Energy management strategy for methanol hybrid...,In order to solve the problem of poor adaptabi...,Abstract\r\nIn order to solve the problem of p...,2025-01-02T14:00:00Z


In [33]:
from transformers import pipeline

# Initialize LLaMA for Sentiment Analysis
def initialize_llama():
    return pipeline("text-classification", model="facebook/bart-large-mnli")

# Sentiment Analysis Function
def analyze_sentiment_with_llama(content, llama_pipeline):
    try:
        # Perform sentiment analysis
        results = llama_pipeline(content)
        return results
    except Exception as e:
        print(f"Error with LLaMA: {e}")
        return None


####  sentimental Analysis for the structured Data

In [34]:
# Initialize the LLaMA pipeline
llama_pipeline = initialize_llama()

# Perform sentiment analysis for all articles in structured data
structured_data['sentiment'] = structured_data['content'].apply(
    lambda content: analyze_sentiment_with_llama(content, llama_pipeline)
)

# Display the results
print(structured_data[['title', 'sentiment']])


Device set to use cpu


                                                title  \
0         AI Dominates 2025 Cybersecurity Predictions   
1   Instant Insights: How To Harness The Power Of ...   
2   How The Tea Industry Can Create More Sustainab...   
3   Moxa router flaws pose serious risks to indust...   
4   NYT Readies Terrain for Incoming Trump Adminis...   
..                                                ...   
95             6. Uphold Voluntarism: Digital Defense   
96  DoubleClickjacking allows clickjacking on majo...   
97  “The bike industry will not get better this ye...   
98  Energy management strategy for methanol hybrid...   
99  Russian media outlets Telegram channels blocke...   

                                            sentiment  
0   [{'label': 'neutral', 'score': 0.9717442989349...  
1   [{'label': 'neutral', 'score': 0.9790660142898...  
2   [{'label': 'neutral', 'score': 0.9885179996490...  
3   [{'label': 'neutral', 'score': 0.6533375978469...  
4   [{'label': 'neutral', 'score': 

#### Risk Analysis 

In [35]:
from textblob import TextBlob

# Define the function for risk analysis using TextBlob
def analyze_risk_with_textblob(content):
    """
    Perform risk analysis on the given content by extracting polarity.
    
    :param content: Text to analyze
    :return: A dictionary with polarity and subjectivity scores
    """
    try:
        blob = TextBlob(content)
        polarity = blob.sentiment.polarity  # Measure of positivity/negativity
        subjectivity = blob.sentiment.subjectivity  # Measure of opinion vs fact
        return {"polarity": polarity, "subjectivity": subjectivity}
    except Exception as e:
        print(f"Error with TextBlob: {e}")
        return None


In [36]:
# Call the Risk Analysis Function for All Articles
for index, row in structured_data.iterrows():
    print(f"Analyzing Risk for Article {index + 1}: {row['title']}")
    risk_analysis = analyze_risk_with_textblob(row['content'])
    print(f"Risk Analysis (Polarity, Subjectivity): {risk_analysis}\n")


Analyzing Risk for Article 1: AI Dominates 2025 Cybersecurity Predictions
Risk Analysis (Polarity, Subjectivity): {'polarity': -0.04999999999999999, 'subjectivity': 0.75}

Analyzing Risk for Article 2: Instant Insights: How To Harness The Power Of Real-Time Data
Risk Analysis (Polarity, Subjectivity): {'polarity': 0.35, 'subjectivity': 0.4}

Analyzing Risk for Article 3: How The Tea Industry Can Create More Sustainable Products
Risk Analysis (Polarity, Subjectivity): {'polarity': 0.4166666666666667, 'subjectivity': 0.4166666666666667}

Analyzing Risk for Article 4: Moxa router flaws pose serious risks to industrial environmets
Risk Analysis (Polarity, Subjectivity): {'polarity': 0.1875, 'subjectivity': 0.37083333333333335}

Analyzing Risk for Article 5: NYT Readies Terrain for Incoming Trump Administration’s Long-Telegraphed Intervention in Mexico
Risk Analysis (Polarity, Subjectivity): {'polarity': 0.030000000000000006, 'subjectivity': 0.27}

Analyzing Risk for Article 6: Musk’s Massi